In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import os

# === OPTION 1: Manual site IDs ===
# site_ids = ["10987", "3133", "10543"]  # Uncomment this line to use manual IDs

# === OPTION 2: Load site IDs from metadata CSV (first 5 STATION_IDs) ===
metadata_path = "meta_data_for_all_active_systems_for_calculation_table_20250401.csv"  # Update this to your metadata CSV path
metadata_df = pd.read_csv(metadata_path, dtype=str)
site_ids = metadata_df["STATION_ID"].head(5).tolist()  # First 5 IDs from the CSV

# Create a list to hold summary results
summary_data = []

# Loop through each site_id_API
for DivrtID in site_ids:
    end_date = datetime.today().strftime("%Y-%m-%d")
    api_url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={DivrtID}&end_date={end_date}&f=json"
    print(f"Fetching data from: {api_url}")

    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()

        if "data" in data:
            df = pd.DataFrame(data["data"], columns=["date", "value"])
            df.rename(columns={"date": "Date", "value": "DISCHARGE"}, inplace=True)

            # Convert data types
            df["DISCHARGE"] = pd.to_numeric(df["DISCHARGE"], errors='coerce')
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
            df.dropna(subset=["Date", "DISCHARGE"], inplace=True)

            # Calculate rate of change (RoC)
            df["RoC"] = df["DISCHARGE"].diff().replace([np.inf, -np.inf], np.nan)
            roc_values = df["RoC"].dropna().abs()

            # Calculate the average rate of change (RoC)
            Average_Rate_Of_Change = roc_values.mean()

            # Calculate the 90th percentile of the (roC) values
            Roc_90th_Percentile = roc_values.quantile(0.90)

            # Minimum values a station can record data
            MinValue = 0

            # IQR test
            filtered_discharge = df["DISCHARGE"][(df["DISCHARGE"] > 0) & (~df["DISCHARGE"].isna())]
            Q1 = filtered_discharge.quantile(0.25)
            Q3 = filtered_discharge.quantile(0.75)
            IQR = Q3 - Q1
            upper_bound = Q3 + 1.5 * IQR

            max_value_iqr_candidates = filtered_discharge[filtered_discharge > upper_bound]
            MaxValue_IQR = max_value_iqr_candidates.min() if not max_value_iqr_candidates.empty else np.nan

            Roc_Standard_Deviation = roc_values.std()
            Discharge_Standard_Deviation = df["DISCHARGE"].std()

            MaxValue_95Perc = filtered_discharge.quantile(0.95)
            MaxValueFinal = min(MaxValue_IQR, MaxValue_95Perc) if not np.isnan(MaxValue_IQR) else MaxValue_95Perc

            summary_data.append({
                "SiteID": DivrtID,
                "MinValue": MinValue,
                "MaxValueFinal": MaxValueFinal,
                "MaxValue_IQR": MaxValue_IQR,
                "MaxValue_95Perc": MaxValue_95Perc,
                "AverageRateOfChange": Average_Rate_Of_Change,
                "UnusualChangeDistribution": Roc_90th_Percentile,
                "UnusualStandardDeviation": float(Roc_Standard_Deviation) if not np.isnan(Roc_Standard_Deviation) else np.nan,
                "DischargeStandardDeviation": float(Discharge_Standard_Deviation) if not np.isnan(Discharge_Standard_Deviation) else np.nan,
            })

        else:
            print("Error: 'data' key not found in response.")
    else:
        print(f"Error fetching data for site {DivrtID}: {response.status_code}")

# Output the results
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv("dvrt_data_qualifiers_max_min_RoC_SD_table.csv", index=False)
print("CSV saved as 'dvrt_data_qualifiers_max_min_RoC_SD_table.csv'")

Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=6&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=18&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=19&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=21&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=26&end_date=2025-04-18&f=json
CSV saved as 'dvrt_data_qualifiers_max_min_RoC_SD_table.csv'
